In [1]:
# # Use this the first time setting up the conda environment
# !pip install mysql-connector-python
# !pip install pymysql
# !pip install ipython-sql pymysql
# !pip install cryptography

import pymysql
import pandas as pd
import hashlib
import csv

In [2]:
### Fill in your user data
user_info = {
    'username': 'rysch01',
    'password': 'SQLP4ssword1!'
}

In [ ]:

# Connect to the MySQL serve
db = pymysql.connect(
    host='localhost',    
    user=user_info['username'],
    password=user_info['password'],
)

cursor = db.cursor()

try: # try to use database; if so, it's already loaded
    cursor.execute("USE cafeteria")
except: # if not, need to load it in next
    cursor.execute('CREATE DATABASE cafeteria;')
    cursor.execute("USE cafeteria")
    ### Define the students table
    declare_students='''
    CREATE TABLE Students (
        pid INT PRIMARY KEY,
        first_name VARCHAR(255),
        last_name VARCHAR(255),
        meal_balance DECIMAL(8, 2),
        to_go_boxes_remaining INT,
        password_hash CHAR(64),
        role CHAR(8)
    );
    '''

    ### Define the menu table
    declare_menu='''
    CREATE TABLE Menu (
        item_id INT PRIMARY KEY,
        item_name VARCHAR(255),
        calories_per_serving INT,
        meal_type CHAR(64),
        available_date DATE,
        price DECIMAL(8, 2)
    );
    '''

    ### Define the transactions table
    declare_transactions='''
    CREATE TABLE Transactions (
        transaction_id INT PRIMARY KEY,
        pid INT,
        item_id INT,
        transaction_type VARCHAR(50),
        transaction_date DATE,
        FOREIGN KEY (pid) REFERENCES Students(pid),
        FOREIGN KEY (item_id) REFERENCES Menu(item_id)
    );
    '''


    cursor.execute(declare_students)
    cursor.execute(declare_menu)
    cursor.execute(declare_transactions)
    db.commit()


    with open('students.csv') as file:
        headers=next(file)
        headers=headers.split(',')
        headers=[h.strip() for h in headers]
        reader=csv.reader(file) 
        info_list=[]
        for r in reader:
            r[-1]=hashlib.sha256(r[-1].encode()).hexdigest()
            # I think we weren't supposed to construct the string this way but cannot remember
            cursor.execute(f'INSERT INTO Students ({", ".join(headers)}) VALUES (%s, %s, %s, %s, %s, %s, %s)', r)

    
    with open('menu.csv') as file:
        headers=next(file)
        headers=headers.split(',')
        headers=[h.strip() for h in headers]
        reader=csv.reader(file) 
        info_list=[]
        for r in reader:
            # I think we weren't supposed to construct the string this way but cannot remember
            cursor.execute(f'INSERT INTO Menu ({", ".join(headers)}) VALUES (%s, %s, %s, %s, %s, %s)', r)

    with open('transactions.csv') as file:
        headers=next(file)
        headers=headers.split(',')
        headers=[h.strip() for h in headers]
        reader=csv.reader(file) 
        info_list=[]
        for r in reader:
            # I think we weren't supposed to construct the string this way but cannot remember
            cursor.execute(f'INSERT INTO Transactions ({", ".join(headers)}) VALUES (%s, %s, %s, %s, %s)', r)

    db.commit()

pid,first_name,last_name,meal_balance,to_go_boxes_remaining,role,password_hash

item_id,item_name,calories_per_serving,meal_type,available_date, price

transaction_id,pid,item_id,transaction_type,transaction_date



In [ ]:
# ## Insert Query Example
# new_item_details = (5, 'Omelette', 600, 'Breakfast', '2024-11-12', 8.00)
# cursor.execute('INSERT INTO Menu (item_id, item_name, calories_per_serving, meal_type, available_date, price) VALUES (%s, %s, %s, %s, %s, %s);', new_item_details)
# db.commit()

# ## Select Query Example
# cursor.execute('SELECT * FROM Menu')
# result = cursor.fetchall()
# print(result)

((1, 'Pasta Primavera', 450, 'lunch', datetime.date(2024, 11, 12), Decimal('11.00')), (2, 'Grilled Chicken Sandwich', 600, 'dinner', datetime.date(2024, 11, 12), Decimal('8.00')), (3, 'Oatmeal', 150, 'breakfast', datetime.date(2024, 11, 12), Decimal('5.00')), (4, 'Vegan Burrito Bowl', 500, 'lunch', datetime.date(2024, 11, 13), Decimal('9.00')), (5, 'Omelette', 600, 'Breakfast', datetime.date(2024, 11, 12), Decimal('8.00')))


In [ ]:
### Be sure to close at the end
db.close()